In [2]:
import numpy as np
from hmmlearn import hmm 


In [3]:
hmm_states = 2

model = hmm.CategoricalHMM(n_components=hmm_states)
model.emissionprob_ = np.array([[0.6,0.4],[0.1,0.9]])
model.transmat_ = np.array([[0.8,0.2],[0.1,0.9]])
model.startprob_ = np.array([0.4,0.6])
data,states = model.sample(n_samples = 1000,random_state=28)
data = np.squeeze(data)

In [4]:
print(states)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0 1
 1 1 1 1 0 0 0 1 1 1 1 0 0 0 0 1 1 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 0 1
 1 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

In [5]:
n_states = 2
n_features = 2
n_obs = data.shape[0]
# emission = np.random.rand(n_obs,n_states)
# emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))

# emission = np.random.rand(n_states,n_features)
# emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))
emission = np.array([[0.2,0.8],[0.3,0.7]])

# transition =  np.random.rand(n_states,n_states)
# transition = transition/np.tile(np.expand_dims(np.sum(transition,axis=1),axis=1),(1,2))
transition = np.array([[0.5,0.5],[0.5,0.5]])


scale_factors = np.zeros((n_obs))
forward_hat = np.zeros((n_obs,n_states))

backward_hat = np.zeros((n_obs,n_states))

init_prob = np.array([0.5,0.5])


p_old = -10000
tol = 0.01
max_iter = 1000

In [6]:
for ite in range(max_iter):
    print(ite)
    forward_hat = np.zeros((n_obs,n_states))
    backward_hat = np.zeros((n_obs,n_states))
    scale_factors = np.zeros((n_obs))


    forward_hat[0,:] = init_prob * emission[:,data[0]]
    scale_factors[0] = np.sum(forward_hat[0,:])
    forward_hat[0,:] = forward_hat[0,:]/scale_factors[0]
    
    for t in range(n_obs-1):
        temp = np.matmul(forward_hat[t,:] ,transition) * emission[:,data[t+1]]
        scale_factors[t+1] = np.sum(temp)
        forward_hat[t+1,:] = temp/scale_factors[t+1]
        # print(f'temp is {temp} and the scale factor is {scale_factors[t+1]} and the forward_hat is {forward_hat[t+1]}')


    backward_hat[-1,:] = scale_factors[-1]
    for t in reversed(range(n_obs-1)):
        temp = np.matmul(backward_hat[t+1,:]*emission[:,data[t+1]],transition.T)
        backward_hat[t,:] = temp/scale_factors[t]
        


    a = np.zeros((n_obs,n_states))
    b = np.zeros((n_obs,n_states,n_states))
    for i in range(n_obs):
        for j in range(n_states):
            a[i,j] = forward_hat[i,j]*backward_hat[i,j]
        temp = np.sum(a[i,:])
        a[i,:] = a[i,:]/temp

    for t in range(n_obs-1):
        for i in range(n_states):
            for j in range(n_states):
                b[t,i,j] = scale_factors[t+1]*forward_hat[t,i]*backward_hat[t+1,j]*transition[i,j]*emission[j,data[t+1]]

    for i in range(n_states):
        for j in range(n_states):
            transition[i,j] = np.sum(b[0:-1,i,j])/np.sum(b[0:-1,i,:])
            # print(np.sum(b[0:-1,i,j]),np.sum(b[0:-1,i,:]))

    for i in range(n_states):
        init_prob[i] = a[0,i]/np.sum(a[0,:])

    for i in range(n_states):
        for j in range(n_states):
            emission[j,i] = np.sum(a[np.argwhere(data==i),j]) / np.sum(a[:,j])
            
    p = np.sum(np.log(scale_factors))
    print(f'p is:{p},transition is {transition},emission is {emission},init is {init_prob}')
    # print(f'transition prob is: {transition}')
    # print(f'emission prob is:{emission}')
    if p>p_old and p - p_old < tol:
        break
    p_old = p

0
p is:-563.4337569074765,transition is [[0.52036151 0.47963849]
 [0.51947354 0.48052646]],emission is [[0.20085356 0.79914644]
 [0.3011197  0.6988803 ]],init is [0.53333333 0.46666667]
1
p is:-563.4384567139621,transition is [[0.54071422 0.45928578]
 [0.53893507 0.46106493]],emission is [[0.20256486 0.79743514]
 [0.30340391 0.69659609]],init is [0.5665318 0.4334682]
2
p is:-563.4352708602925,transition is [[0.5610682 0.4389318]
 [0.5583987 0.4416013]],emission is [[0.20429423 0.79570577]
 [0.30575078 0.69424922]],init is [0.59944396 0.40055604]
